# B1: FBA demo

# Introduction

This notebook shows how to use jQMM for FBA (Flux Balance Analysis) and FVA (Flux Variability Analysis) by reproducing results in table 2 from Becker *et al* 2007.

# Setup

First, we need to set the path and environment variable properly:

In [1]:
quantmodelDir = '/users/hgmartin/libraries/quantmodel'

This is the only place where the jQMM library path needs to be set.

In [2]:
%matplotlib inline

import sys, os
pythonPath = quantmodelDir+"/code/core"
if pythonPath not in sys.path:
    sys.path.append(pythonPath)
os.environ["QUANTMODELPATH"] = quantmodelDir

Then load all needed packages:

In [3]:
%matplotlib inline
import FluxModels

And change to the tests directory (no need for the scratch directory for this one):

In [4]:
cd ~/tests

/mnt/nfs.jbei/nfs/users/hgmartin/tests


# Results

First we load the sbml file into FBA model instances:

In [5]:
fileName = os.environ["QUANTMODELPATH"]+'/data/sbmlFiles/EciJR904.xml'
FBAmodel  = FluxModels.FBAModel(fileName)
FBAmodel2 = FluxModels.FBAModel(fileName)
FBAmodel3 = FluxModels.FBAModel(fileName)


Then we change the flux bounds for model 2 to grow on succinate instead of glucose and for model 3 to grow anaerobically:

In [6]:
FBAmodel2.reactionNetwork.changeFluxBounds('EX_glc_e_'  ,0)
FBAmodel2.reactionNetwork.changeFluxBounds('EX_succ_e_' ,-9)

FBAmodel3.reactionNetwork.changeFluxBounds('EX_o2_e_'  ,0)       

Once flux bounds are properly set we can calculate the corresponding fluxes for each model:

In [7]:
FBAresults   = FBAmodel.findFluxes()
FBAresults2  = FBAmodel2.findFluxes()
FBAresults3  = FBAmodel3.findFluxes()

and obtain a reaction dictionary for the reaction network (see notebook A3):

In [9]:
rxnDict  = FBAresults.reactionNetwork.reactionList.getReactionDictionary()
rxnDict2 = FBAresults2.reactionNetwork.reactionList.getReactionDictionary()
rxnDict3 = FBAresults3.reactionNetwork.reactionList.getReactionDictionary()

The net flux (the only one calculated for FBA, as opposed to $^{13}$C MFA) of exchange flux for e.g. glucose input, pyruvate dehydrogenase (PDH), and glucose-6-phosphate isomerase (PGI) are:

In [10]:
print 'EX_glc_e_:\t '+str(rxnDict['EX_glc_e_'].flux.net)
print 'PDH:\t\t '+str(rxnDict['PDH'].flux.net)
print 'PGI:\t\t '+str(rxnDict['PGI'].flux.net)

EX_glc_e_:	 -6.0
PDH:		 5.02097213844
PGI:		 3.12895634753


and the growth rates shown in table 2 in Becker *et al* 2007 are:

In [11]:
print rxnDict['BiomassEcoli'].flux.net

0.53911307865


for glucose aerobic growth, which compares well to the value of 0.539 in table 2 from Becker *et al* 2007.

In [12]:
print rxnDict2['BiomassEcoli'].flux.net

0.385924559398


for succinate aerobic growth, which compares well to the value of 0.386 in table 2 from Becker *et al* 2007.

In [13]:
print rxnDict3['BiomassEcoli'].flux.net

0.0967170799042


for glucose anaerobic growth, which compares well to the value of 0.097 in table 2 from Becker *et al* 2007.

# References

Becker, Scott A., et al. "Quantitative prediction of cellular metabolism with constraint-based models: the COBRA Toolbox." Nature protocols 2.3 (2007): 727-738.